In [12]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Activation, GRU, Input, Dropout, BatchNormalization, Dense, concatenate, LSTM
import tensorflow.keras as keras
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

In [13]:
data = pd.read_csv('data/HY_cleaned.csv')
'''
(Currently perdicting:
1. Payment (mean)
2. Number_of_observations
3. Payment_std
4. Payment_min
5. Payment_max
6. Null probability
'''

class CONFIG:
    LOOK_WINDOW = 3
    INPUT_SHAPE = (LOOK_WINDOW,6)
    DATA_DIR = 'data/'

In [14]:
def prepare_data(data):
    payments = data.Payment
    d = data[['Payment','Number_of_observations','Payment_std','Payment_min','Payment_max']]
    nulls = payments.isnull()
    out = np.zeros((len(payments),6))

    for i in range(len(payments)):
        valid = [payments[i]/1e9,d.Number_of_observations[i],d.Payment_std[i]/1e9,d.Payment_min[i]/1e9,d.Payment_max[i]/1e9,1]
        non_valid = [0,0,0,0,0,0]

        if(nulls[i]):
            out[i] = non_valid
        else:
            out[i] = valid


    return out

In [15]:
prep_data = prepare_data(data)

In [16]:
prep_data = prep_data[:-1]  #getting the len to be divisible by 4 = (LOOK_WINDOW + 1)

In [17]:
num_samples = int(len(prep_data)/(CONFIG.LOOK_WINDOW+1))
x_train = np.zeros((num_samples,3,6))
y_train = np.zeros((num_samples,6))

In [18]:
for i in range(num_samples):
    x_out = []
    for j in range(CONFIG.LOOK_WINDOW):
        x_out.append(list(prep_data[4*i+j]))
    x_train[i] = np.asarray(x_out)
    y_train[i] = np.asarray(list(prep_data[4*i + CONFIG.LOOK_WINDOW]))



x_train = np.asarray(x_train,dtype=np.float32)
y_train = np.asarray(y_train,dtype=np.float32)

In [19]:
def CrossEntropy(yHat, y):
    loss = 0
    for pred,act in zip(yHat,y):
        if act == 1:
          loss += -tf.math.log(pred)
        else:
          loss += -tf.math.log(1 - pred)

    return loss/len(y)


def custom_loss(model,x,y):
    #assert(model.output.shape == OUTPUT_SHAPE)
    alpha=100
    beta=1
    y_pred = model(x)
    nan_loss = CrossEntropy(y_pred[:,5],y[:,5])
    payments_loss = tf.reduce_mean(tf.math.square(y[:,:-1] - y_pred[:,:-1]))
    final_loss = alpha*nan_loss + beta*payments_loss
    print("final loss: {} nan_loss: {} payments_loss:{}".format(final_loss,alpha*nan_loss,beta*payments_loss))
    return final_loss


def compute_gradients(model,x,y):
    with tf.GradientTape() as tape:
        loss = custom_loss(model,x,y)
        gradients = tape.gradient(loss,model.trainable_variables)

    return gradients, loss

def apply_gradients(optimizer,gradients,variables):
    optimizer.apply_gradients(zip(gradients,variables))


def base_model(input_shape):

    x_input = Input(shape = input_shape)
    x = GRU(units=128,return_sequences = False)(x_input)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = GRU(units=64,return_sequences = False)(x_input)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(8,activation='relu')(x)
    x = Dense(6,activation='relu')(x)
    y_payment = Dense(5,activation='relu')(x)
    y_prob = Dense(1,activation='sigmoid')(x)

    x = concatenate([y_payment,y_prob])



    return keras.models.Model(inputs = x_input, outputs=x)

In [9]:
model = base_model(CONFIG.INPUT_SHAPE)

In [10]:
epochs = 10000
learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate)

for i in range(1, epochs+1):
    gradients, loss = compute_gradients(model,x_train[:400],y_train[:400])
    apply_gradients(optimizer,gradients,model.trainable_variables)

final loss: 1635.5711669921875 nan_loss: 6.934683322906494 payments_loss:1628.636474609375
final loss: 1635.56494140625 nan_loss: 6.933181285858154 payments_loss:1628.6317138671875
final loss: 1635.5584716796875 nan_loss: 6.9313812255859375 payments_loss:1628.6270751953125
final loss: 1635.551513671875 nan_loss: 6.92929220199585 payments_loss:1628.6221923828125
final loss: 1635.544677734375 nan_loss: 6.926912784576416 payments_loss:1628.6177978515625
final loss: 1635.5372314453125 nan_loss: 6.924319267272949 payments_loss:1628.6129150390625
final loss: 1635.5294189453125 nan_loss: 6.921640396118164 payments_loss:1628.6077880859375
final loss: 1635.521484375 nan_loss: 6.9188232421875 payments_loss:1628.6026611328125
final loss: 1635.5128173828125 nan_loss: 6.915853500366211 payments_loss:1628.596923828125
final loss: 1635.504150390625 nan_loss: 6.912841796875 payments_loss:1628.59130859375
final loss: 1635.4952392578125 nan_loss: 6.90971565246582 payments_loss:1628.5855712890625
final l

final loss: 1634.56005859375 nan_loss: 6.533292770385742 payments_loss:1628.0267333984375
final loss: 1634.5440673828125 nan_loss: 6.526146411895752 payments_loss:1628.0179443359375
final loss: 1634.5289306640625 nan_loss: 6.519173622131348 payments_loss:1628.009765625
final loss: 1634.513671875 nan_loss: 6.512295722961426 payments_loss:1628.0013427734375
final loss: 1634.4981689453125 nan_loss: 6.505380153656006 payments_loss:1627.9927978515625
final loss: 1634.4830322265625 nan_loss: 6.498538017272949 payments_loss:1627.9844970703125
final loss: 1634.46728515625 nan_loss: 6.491508483886719 payments_loss:1627.975830078125
final loss: 1634.4517822265625 nan_loss: 6.484453201293945 payments_loss:1627.96728515625
final loss: 1634.4364013671875 nan_loss: 6.477294921875 payments_loss:1627.9591064453125
final loss: 1634.4207763671875 nan_loss: 6.470148086547852 payments_loss:1627.95068359375
final loss: 1634.4051513671875 nan_loss: 6.462870121002197 payments_loss:1627.9422607421875
final lo

final loss: 1633.27392578125 nan_loss: 5.982851982116699 payments_loss:1627.291015625
final loss: 1633.2593994140625 nan_loss: 5.977513790130615 payments_loss:1627.2818603515625
final loss: 1633.2451171875 nan_loss: 5.9721856117248535 payments_loss:1627.27294921875
final loss: 1633.231201171875 nan_loss: 5.96687126159668 payments_loss:1627.2642822265625
final loss: 1633.216552734375 nan_loss: 5.961564064025879 payments_loss:1627.2550048828125
final loss: 1633.2021484375 nan_loss: 5.956295013427734 payments_loss:1627.245849609375
final loss: 1633.187744140625 nan_loss: 5.951039791107178 payments_loss:1627.2366943359375
final loss: 1633.17333984375 nan_loss: 5.945792198181152 payments_loss:1627.2275390625
final loss: 1633.1588134765625 nan_loss: 5.940551280975342 payments_loss:1627.21826171875
final loss: 1633.14453125 nan_loss: 5.935302734375 payments_loss:1627.209228515625
final loss: 1633.1297607421875 nan_loss: 5.930051326751709 payments_loss:1627.19970703125
final loss: 1633.1153564

final loss: 1604.24755859375 nan_loss: 6.297096252441406 payments_loss:1597.950439453125
final loss: 1603.2894287109375 nan_loss: 6.307712554931641 payments_loss:1596.981689453125
final loss: 1602.3062744140625 nan_loss: 6.319850921630859 payments_loss:1595.9864501953125
final loss: 1601.2843017578125 nan_loss: 6.332597255706787 payments_loss:1594.95166015625
final loss: 1600.239990234375 nan_loss: 6.347489833831787 payments_loss:1593.8924560546875
final loss: 1599.166748046875 nan_loss: 6.363001346588135 payments_loss:1592.8037109375
final loss: 1598.0654296875 nan_loss: 6.380919456481934 payments_loss:1591.6845703125
final loss: 1596.95068359375 nan_loss: 6.398621082305908 payments_loss:1590.552001953125
final loss: 1595.81884765625 nan_loss: 6.418154716491699 payments_loss:1589.400634765625
final loss: 1594.6708984375 nan_loss: 6.440132141113281 payments_loss:1588.230712890625
final loss: 1593.506103515625 nan_loss: 6.464752197265625 payments_loss:1587.0413818359375
final loss: 1592

final loss: 1500.720947265625 nan_loss: 11.602621078491211 payments_loss:1489.1182861328125
final loss: 1499.781494140625 nan_loss: 11.648770332336426 payments_loss:1488.1326904296875
final loss: 1498.8428955078125 nan_loss: 11.694794654846191 payments_loss:1487.1480712890625
final loss: 1497.9049072265625 nan_loss: 11.740618705749512 payments_loss:1486.164306640625
final loss: 1496.9671630859375 nan_loss: 11.78641128540039 payments_loss:1485.1807861328125
final loss: 1496.0294189453125 nan_loss: 11.832209587097168 payments_loss:1484.197265625
final loss: 1495.094482421875 nan_loss: 11.877884864807129 payments_loss:1483.216552734375
final loss: 1494.1602783203125 nan_loss: 11.92355728149414 payments_loss:1482.2366943359375
final loss: 1493.2274169921875 nan_loss: 11.969377517700195 payments_loss:1481.258056640625
final loss: 1492.29638671875 nan_loss: 12.015223503112793 payments_loss:1480.2811279296875
final loss: 1491.3671875 nan_loss: 12.061174392700195 payments_loss:1479.30603027343

...

final loss: 27.7801456451416 nan_loss: 0.4082028567790985 payments_loss:27.3719425201416
final loss: 27.7777099609375 nan_loss: 0.4084695875644684 payments_loss:27.369239807128906
final loss: 27.773839950561523 nan_loss: 0.40837305784225464 payments_loss:27.365467071533203
final loss: 27.76799201965332 nan_loss: 0.40848955512046814 payments_loss:27.3595027923584
final loss: 27.76242446899414 nan_loss: 0.40833818912506104 payments_loss:27.35408592224121
final loss: 27.759132385253906 nan_loss: 0.40860092639923096 payments_loss:27.35053062438965
final loss: 27.756574630737305 nan_loss: 0.4088173508644104 payments_loss:27.34775733947754
final loss: 27.7535343170166 nan_loss: 0.408382773399353 payments_loss:27.345151901245117
final loss: 27.75130271911621 nan_loss: 0.40880894660949707 payments_loss:27.342493057250977
final loss: 27.7471866607666 nan_loss: 0.40856629610061646 payments_loss:27.338621139526367
final loss: 27.743135452270508 nan_loss: 0.4089282751083374 payments_loss:27.334207

final loss: 27.457365036010742 nan_loss: 0.40536943078041077 payments_loss:27.0519962310791
final loss: 27.453258514404297 nan_loss: 0.4053575098514557 payments_loss:27.047901153564453
final loss: 27.447240829467773 nan_loss: 0.40535426139831543 payments_loss:27.041887283325195
final loss: 27.439491271972656 nan_loss: 0.4051918387413025 payments_loss:27.034299850463867
final loss: 27.43292236328125 nan_loss: 0.4053772985935211 payments_loss:27.027545928955078
final loss: 27.429813385009766 nan_loss: 0.4052225351333618 payments_loss:27.02459144592285
final loss: 27.428476333618164 nan_loss: 0.4050745964050293 payments_loss:27.023401260375977
final loss: 27.42692756652832 nan_loss: 0.4050537943840027 payments_loss:27.021873474121094
final loss: 27.423704147338867 nan_loss: 0.40527865290641785 payments_loss:27.01842498779297
final loss: 27.42017936706543 nan_loss: 0.404788613319397 payments_loss:27.015390396118164
final loss: 27.417949676513672 nan_loss: 0.40526631474494934 payments_loss:

final loss: 27.115524291992188 nan_loss: 0.40224358439445496 payments_loss:26.713281631469727
final loss: 27.111591339111328 nan_loss: 0.40161535143852234 payments_loss:26.709976196289062
final loss: 27.10751724243164 nan_loss: 0.4023936986923218 payments_loss:26.705123901367188
final loss: 27.10163688659668 nan_loss: 0.4019857943058014 payments_loss:26.69965171813965
final loss: 27.0958251953125 nan_loss: 0.4021097719669342 payments_loss:26.693716049194336
final loss: 27.092029571533203 nan_loss: 0.40177613496780396 payments_loss:26.69025421142578
final loss: 27.089229583740234 nan_loss: 0.40164321660995483 payments_loss:26.687585830688477
final loss: 27.086257934570312 nan_loss: 0.40199214220046997 payments_loss:26.68426513671875
final loss: 27.082244873046875 nan_loss: 0.40170910954475403 payments_loss:26.6805362701416
final loss: 27.078922271728516 nan_loss: 0.40207433700561523 payments_loss:26.676847457885742
final loss: 27.074920654296875 nan_loss: 0.40150880813598633 payments_lo

final loss: 26.773727416992188 nan_loss: 0.39931344985961914 payments_loss:26.374414443969727
final loss: 26.77117919921875 nan_loss: 0.39830219745635986 payments_loss:26.37287712097168
final loss: 26.766855239868164 nan_loss: 0.39948731660842896 payments_loss:26.367368698120117
final loss: 26.761241912841797 nan_loss: 0.39862141013145447 payments_loss:26.362621307373047
final loss: 26.75513458251953 nan_loss: 0.3988761305809021 payments_loss:26.356258392333984
final loss: 26.750244140625 nan_loss: 0.3985373377799988 payments_loss:26.351707458496094
final loss: 26.7451171875 nan_loss: 0.3989022672176361 payments_loss:26.346214294433594
final loss: 26.74058723449707 nan_loss: 0.39912283420562744 payments_loss:26.34146499633789
final loss: 26.736143112182617 nan_loss: 0.3986598551273346 payments_loss:26.337482452392578
final loss: 26.733135223388672 nan_loss: 0.39885246753692627 payments_loss:26.33428192138672
final loss: 26.730247497558594 nan_loss: 0.39834585785865784 payments_loss:26.

final loss: 26.41974449157715 nan_loss: 0.39625871181488037 payments_loss:26.02348518371582
final loss: 26.41756820678711 nan_loss: 0.39603325724601746 payments_loss:26.021535873413086
final loss: 26.416885375976562 nan_loss: 0.3960525691509247 payments_loss:26.020832061767578
final loss: 26.414636611938477 nan_loss: 0.39595669507980347 payments_loss:26.018680572509766
final loss: 26.40928077697754 nan_loss: 0.3963952660560608 payments_loss:26.01288604736328
final loss: 26.40239143371582 nan_loss: 0.39593803882598877 payments_loss:26.006452560424805
final loss: 26.395652770996094 nan_loss: 0.3960612416267395 payments_loss:25.999591827392578
final loss: 26.391460418701172 nan_loss: 0.39602264761924744 payments_loss:25.995437622070312
final loss: 26.388641357421875 nan_loss: 0.3960627019405365 payments_loss:25.992578506469727
final loss: 26.388790130615234 nan_loss: 0.39563122391700745 payments_loss:25.9931583404541
final loss: 26.388696670532227 nan_loss: 0.39576268196105957 payments_lo

In [11]:
#model.save('six_features_10000.h5')

In [23]:
model = keras.models.Model.load('six_features_10000.h5') 

AttributeError: type object 'Model' has no attribute 'load'

In [12]:
y_preds = model.predict(x_train[400:])

In [13]:
y_preds.shape

(154, 6)

In [14]:
y_train[400:].shape

(154, 6)

In [15]:
y_preds[:10]

array([[2.20325041e+00, 6.62393112e+01, 2.64590168e+00, 0.00000000e+00,
        1.21208897e+01, 9.99860704e-01],
       [2.00055408e+00, 4.83765144e+01, 2.42302799e+00, 0.00000000e+00,
        1.22304058e+01, 9.99017656e-01],
       [2.52334404e+00, 6.42600784e+01, 3.37900639e+00, 0.00000000e+00,
        1.41260605e+01, 9.98820186e-01],
       [2.73156881e+00, 6.93678131e+01, 3.76468754e+00, 0.00000000e+00,
        1.53244724e+01, 9.99968171e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        4.23104107e-01, 1.33964419e-03],
       [2.24202490e+00, 2.36047020e+01, 3.50472593e+00, 0.00000000e+00,
        1.08775558e+01, 9.85429406e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        4.13152844e-01, 1.54080987e-03],
       [1.95330286e+00, 2.01977264e+02, 1.63697910e+00, 0.00000000e+00,
        1.18833466e+01, 1.00000000e+00],
       [8.54782164e-01, 2.30448742e+01, 1.41200989e-01, 0.00000000e+00,
        6.21665525e+00, 

In [16]:
y_train[400:][:10]

array([[2.21522474e+00, 4.30000000e+01, 2.34728050e+00, 2.15107407e-02,
        1.07650003e+01, 1.00000000e+00],
       [1.54328299e+00, 8.80000000e+01, 1.85214686e+00, 2.14559995e-02,
        1.02092590e+01, 1.00000000e+00],
       [1.05949998e+00, 1.00000000e+00,            nan, 1.05949998e+00,
        1.05949998e+00, 1.00000000e+00],
       [2.80522823e+00, 9.20000000e+01, 3.44245911e+00, 1.05999745e-04,
        1.17700005e+01, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [1.71958125e+00, 3.70000000e+01, 1.69960415e+00, 1.06829748e-01,
        9.50875378e+00, 1.00000000e+00],
       [1.86784029e+00, 1.20000000e+01, 1.66358018e+00, 7.27057680e-02,
        5.23687506e+00, 1.00000000e+00],
       [1.82215762e+00, 2.90000000e+01, 2.21269941e+00, 4.08167504e-02,
        1.04168301e+01, 1.00000000e+00],
       [1.75579596e+00, 4.50000000e+01, 1.87125742e+00, 5.35275005e-02,
        7.60339022e+00, 